In [1]:
# Load MNIST data set

from tensorflow.keras.datasets import mnist
train_set, test_set = mnist.load_data()

In [ ]:
# generate a simple NN model

from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np

    
kernel_initializer = 'he_normal'
activation = "relu"

def get_model(chs=256):
    shape=(28,28,1)
    
    inputs = Input(shape)
    layer = Flatten()

In [10]:
# generate a simple NN model

from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np

    
kernel_initializer = 'he_normal'
activation = "relu"

def get_model(chs=256):
    shape=(28,28,1)
    
    inputs = Input(shape)
    layer = Flatten()(inputs)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    output = Dense(10, activation='linear', use_bias=True, kernel_initializer=kernel_initializer)(layer)

    model = Model(inputs, output)
    return model

model = get_model()


<tf.Variable 'dense_9/kernel:0' shape=(784, 256) dtype=float32, numpy=
array([[-0.02880584,  0.04084021,  0.05664704, ...,  0.01978791,
         0.03381465, -0.00479045],
       [-0.03906069, -0.07141214, -0.02805506, ...,  0.0266929 ,
         0.04586157,  0.07790492],
       [ 0.05221974,  0.00223555, -0.00929282, ..., -0.06812875,
        -0.00967157,  0.0981956 ],
       ...,
       [-0.07523067, -0.04610952,  0.04351881, ..., -0.00605765,
        -0.00031668, -0.00746796],
       [-0.00727552,  0.07479773,  0.00788088, ...,  0.00595186,
         0.0009602 ,  0.05531799],
       [ 0.06108871, -0.02102383,  0.04120374, ..., -0.01849575,
        -0.07379042, -0.0147734 ]], dtype=float32)>

In [62]:
def calculate_entropy(variables):
    # TODO implement
    # here you need to also output the reange of the compressed data (minimum, maximum)
    flat_vars = variables.flatten()
    hist, bins = np.histogram(flat_vars, bins='auto')
    probs = hist / np.sum(hist)
    probs[probs == 0] = 1  # replace zeros with ones
    entropy = -np.sum(probs * np.log2(probs))
    return entropy, (np.min(flat_vars), np.max(flat_vars))

class CompressibleNN(keras.Model):
    def __init__(self, net_model):
        super(CompressibleNN, self).__init__()
        self.net_model = net_model
        self.CE_loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    def call(self, inputs):
        return self.net_model(inputs)
    
    def entropy_loss(self, inputs):
        # TODO implemment the entropy calculation here for each layer parameters
        # for each layer: calculate a histogram and from then calculate the entropy
        # return a sum of all entropies
        
        entropy = 0
        # for l in self.net_model.layers:
        #     #TODO: if layer is dense:
        #     for v in l.trainable_variables:
        #         # here you have a set of variable for which you can calculate the histogram and entropy
        #         entropy += calculate_entropy(v)
        # return 0
        
        for l in self.net_model.layers:
            if isinstance(l, keras.layers.Dense):
                for v in l.trainable_variables:
                    v_entropy, v_range = calculate_entropy(v.numpy().flatten())
                    entropy += v_entropy
            
        return entropy

    def train_step(self, input):
        images = input[0]
        labels = input[1]

        with tf.GradientTape() as tape:
            output = self.net_model(images)
            loss = self.CE_loss(labels, output) + \
                    self.entropy_loss(images)

        # Get the gradients w.r.t the loss
        gradient = tape.gradient(loss, self.net_model.trainable_variables)
        # Update the weights using the generator optimizer
        self.optimizer.apply_gradients(
            zip(gradient, self.net_model.trainable_variables)
        )
        return {"loss": loss}
    
compNN = CompressibleNN(model)
optimizer = tf.optimizers.Adam(learning_rate=1e-3, beta_1=0.9)
compNN.compile(optimizer, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [63]:
# train the model

compNN.fit(x=train_set[0], y=train_set[1], epochs=1, batch_size=32)

1875/1875 [==============================] - 3s 2ms/step - loss: 0.1501


In [53]:
# test performance
res = model(test_set[0])
print("test acc: ", 100*(np.argmax(res, axis=1)==test_set[1]).mean())

test acc:  95.32000000000001
